## Skin Cancer Classification Transfer Learning VGG16

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import sklearn

### Creating Dataset

In [ ]:

import os

# Your base path in Drive
base_path = "/content/drive/MyDrive/btech_project"

# Folder where you want to store organized dataset
dataset_base = os.path.join(base_path, "HAM_Dataset")

# Create main folder if not exists
os.makedirs(dataset_base, exist_ok=True)

# Class names
classes = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"]

# Create subfolders
for cls in classes:
    class_path = os.path.join(dataset_base, cls)
    os.makedirs(class_path, exist_ok=True)

print("All folders created successfully!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import cv2
import matplotlib.pyplot as plt
import os

images_dir_1 = "/content/drive/MyDrive/btech_project/HAM10000_images_part_1"
images_dir_2 = "/content/drive/MyDrive/btech_project/HAM10000_images_part_2"

# Combine image lists
all_images = (
    [os.path.join(images_dir_1, img) for img in os.listdir(images_dir_1)]
    +
    [os.path.join(images_dir_2, img) for img in os.listdir(images_dir_2)]
)

# Pick first image from combined list
img_path = all_images[0]
skin = cv2.imread(img_path)

plt.imshow(cv2.cvtColor(skin, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

print("Image shape:", skin.shape)


In [ ]:

import pandas as pd
import os

base_path = "/content/drive/MyDrive/btech_project"
metadata_path = f"{base_path}/HAM10000_metadata.csv"

# Read metadata
df_labels = pd.read_csv(metadata_path)

# Map dx → folder label
dx_to_label = {
    "mel": "MEL",
    "nv": "NV",
    "bcc": "BCC",
    "akiec": "AKIEC",
    "bkl": "BKL",
    "df": "DF",
    "vasc": "VASC"
}

df_labels["label"] = df_labels["dx"].map(dx_to_label)

# Set image_id as index so we can do df_labels.loc[image_id, "label"]
df_labels.set_index("image_id", inplace=True)

df_labels.head()


In [ ]:

import numpy as np
from sklearn.utils import class_weight

classes = np.array(['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC'])

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=df_labels["label"].values
)

class_weights



In [ ]:
# # classes array defines YOUR intended order
classes = np.array(['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC'])

class_wt_dict = {
    train_gen.class_indices[label]: weight
    for label, weight in zip(classes, class_weights)
}

class_wt_dict


In [ ]:
import os
import shutil
from tqdm import tqdm

base_path = "/content/drive/MyDrive/btech_project"

images_dir_1 = os.path.join(base_path, "HAM10000_images_part_1")
images_dir_2 = os.path.join(base_path, "HAM10000_images_part_2")

dataset_dir = os.path.join(base_path, "HAM_Dataset")

# Combine image file lists from both folders
all_images = (
    [os.path.join(images_dir_1, img) for img in os.listdir(images_dir_1)]
    +
    [os.path.join(images_dir_2, img) for img in os.listdir(images_dir_2)]
)

print("Total entries found:", len(all_images))

skipped_not_jpg = 0
skipped_no_label = 0

for img_path in tqdm(all_images):
    image = os.path.basename(img_path)  # e.g., ISIC_003412.jpg or .ipynb_checkpoints

    # 1) Skip anything that is not a .jpg file
    if not image.lower().endswith(".jpg"):
        skipped_not_jpg += 1
        continue

    # 2) Remove extension → get image_id
    fname = os.path.splitext(image)[0]   # ISIC_003412

    # 3) Skip if this image_id is not in df_labels index
    if fname not in df_labels.index:
        skipped_no_label += 1
        # Optional: print once or log
        # print("No label for:", fname)
        continue

    label = df_labels.loc[fname, "label"]    # MEL / NV / ...

    dst = os.path.join(dataset_dir, label, image)
    shutil.copyfile(img_path, dst)

print("Skipped non-JPG entries:", skipped_not_jpg)
print("Skipped files with no label:", skipped_no_label)


### Train Test Split

In [ ]:
!cp -r "/content/drive/MyDrive/btech_project/HAM_Dataset" "/content/HAM_Dataset"


In [ ]:
!find /content/HAM_Dataset -type f -size 0

In [ ]:
import os
bad_file = "/content/HAM_Dataset/NV/ISIC_0029218.jpg"

if os.path.exists(bad_file):
    os.remove(bad_file)
    print(f"✅ Deleted corrupted file: {bad_file}")
else:
    print("File already deleted.")

In [ ]:

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample  # <--- Required for balancing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocess

base_path = "/content/drive/MyDrive/btech_project"
dataset_dir = "/content/HAM_Dataset"

# 1. PREPARE DATAFRAME
# df_labels has index = image_id and column "label"
df = df_labels.copy().reset_index()
df.rename(columns={df.columns[0]: "image_id"}, inplace=True)

# Build full filepaths
df["filepath"] = df.apply(
    lambda r: os.path.join(dataset_dir, r["label"], r["image_id"] + ".jpg"),
    axis=1
)

# 2. SPLIT (Stratified to keep ratios in Test/Val consistent)
# 80% train, 20% temp
train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# 10% val, 10% test
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["label"],
    random_state=42
)

print(f"Original Train Size: {len(train_df)}")
print("Original Class Counts:\n", train_df['label'].value_counts())


# Find the maximum count (usually NV class)
max_samples = train_df['label'].value_counts().max()
print(f"\nUpsampling all classes to {max_samples} samples each...")

balanced_dfs = []

# Loop through each class (0 to 6)
for label_code in train_df['label'].unique():
    # Isolate the specific class
    class_df = train_df[train_df['label'] == label_code]

    # Resample (Duplicate rows) until it matches max_samples
    class_df_upsampled = resample(
        class_df,
        replace=True,     # Sample with replacement (duplicates allowed)
        n_samples=max_samples,
        random_state=42
    )
    balanced_dfs.append(class_df_upsampled)

# Combine back into a single dataframe
train_df_balanced = pd.concat(balanced_dfs)

# Shuffle the rows so classes aren't clumped together
train_df_balanced = train_df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"New Balanced Train Size: {len(train_df_balanced)}")
print("New Class Counts:\n", train_df_balanced['label'].value_counts())


batch_size = 16
target_size = (224, 224)
train_datagen = ImageDataGenerator(
    preprocessing_function=eff_preprocess,
    rotation_range=20,       # ✅ ENABLED
    width_shift_range=0.1,   # ✅ ENABLED
    height_shift_range=0.1,  # ✅ ENABLED
    shear_range=0.1,         # ✅ ENABLED
    zoom_range=0.1,          # ✅ ENABLED
    horizontal_flip=True,    # ✅ ENABLED
    fill_mode='nearest'
)


test_val_datagen = ImageDataGenerator(
    preprocessing_function=eff_preprocess
)


train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df_balanced,
    x_col="filepath",
    y_col="label",
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

val_gen = test_val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

test_gen = test_val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

print("Class indices:", train_gen.class_indices)

In [ ]:

bad_id = "ISIC_0029218"

# 1. Remove from standard splits (just in case)
train_df = train_df[train_df['image_id'] != bad_id]
val_df = val_df[val_df['image_id'] != bad_id]
test_df = test_df[test_df['image_id'] != bad_id]


if 'train_df_balanced' in locals():
    old_len = len(train_df_balanced)
    train_df_balanced = train_df_balanced[train_df_balanced['image_id'] != bad_id]
    print(f"Removed {old_len - len(train_df_balanced)} copies of bad file from train_df_balanced.")

print(f"Cleaned DataFrames. Final Training Size: {len(train_df_balanced)}")


train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df_balanced,
    x_col="filepath",
    y_col="label",
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
print("Train samples:", train_gen.samples)
print("Batch size:", train_gen.batch_size)
print("Steps per epoch:", train_gen.samples // train_gen.batch_size)

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

import time

start = time.time()
batch_x, batch_y = next(train_gen)
print("Time to load + preprocess one batch:", time.time() - start, "seconds")



##     

## Transfer Learning using VGG16 Model

In [ ]:

classes = np.array(['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC'])

class_wt_dict = {
    train_gen.class_indices[label]: weight
    for label, weight in zip(classes, class_weights)
}

class_wt_dict


## Model Training

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import top_k_categorical_accuracy

base_path = "/content/drive/MyDrive/btech_project"

# 1. Base model
image_size = 224
num_classes = 7

base_model = EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size, image_size, 3)
)

# 2. Add head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# 3. Freeze all backbone layers initially
for layer in base_model.layers:
    layer.trainable = False

last_n = 20
last_20_layers = base_model.layers[-last_n:]

print("\nUnfreezing these last 20 layers (excluding BN):")
for layer in last_20_layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = False     # ❌ keep BN frozen
    else:
        layer.trainable = True      # ✅ unfreeze
    print(f"{layer.name} → trainable={layer.trainable}")

# 5. Top-k metrics
def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

# 6. Compile for EfficientNet fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy", top_2_accuracy, top_3_accuracy]
)

# 7. Mild class weights (example)
# class_weights = {
#     0: 1.0, 1: 1.0, 2: 1.0,
#     3: 1.0, 4: 1.7, 5: 1.0, 6: 1.0
# }

# 8. Callbacks
checkpoint = ModelCheckpoint(
    filepath=os.path.join(base_path, "eff_fe_ov1.keras"),
    monitor='val_categorical_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_categorical_accuracy',
    factor=0.5,
    patience=2,
    verbose=1,
    mode='max',
    min_lr=1e-5
)

callbacks_list = [checkpoint, reduce_lr]

# 9. Train (batch_size = 32 from your generator)
epochs = 30

history = model.fit(
    train_gen,
    epochs=epochs,
    validation_data=val_gen,
    # class_weight=class_weights,
    callbacks=callbacks_list,
    verbose=1
)

# 10. Evaluation
val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = model.evaluate(val_gen)
print("val_loss:", val_loss)
print("val_cat_acc:", val_cat_acc)
print("val_top_2_acc:", val_top_2_acc)
print("val_top_3_acc:", val_top_3_acc)


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import top_k_categorical_accuracy

base_path = "/content/drive/MyDrive/btech_project"
saved_model_path = os.path.join(base_path, "eff_fe_ov1_phase2.keras")
new_checkpoint_path = os.path.join(base_path, "eff_fe_ov1_phase3.keras")


def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)


print(f"Loading model from: {saved_model_path}...")

model = load_model(
    saved_model_path,
    custom_objects={
        'top_2_accuracy': top_2_accuracy,
        'top_3_accuracy': top_3_accuracy
    }
)


checkpoint = ModelCheckpoint(
    filepath=new_checkpoint_path,      # Overwrite the same file if it improves
    monitor='val_categorical_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_categorical_accuracy',
    factor=0.5,
    patience=2,
    verbose=1,
    mode='max',
    min_lr=1e-5
)

callbacks_list = [checkpoint, reduce_lr]


total_epochs = 30  # The total goal
start_epoch = 17  # You finished 11 epochs (0-10), so start at index 11 (Epoch 12)

print(f"Resuming training from Epoch {start_epoch + 1} to {total_epochs}...")

history = model.fit(
    train_gen,
    epochs=total_epochs,
    initial_epoch=start_epoch,
    validation_data=val_gen,
    callbacks=callbacks_list,
    verbose=1
)


val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = model.evaluate(val_gen)
print("val_loss:", val_loss)
print("val_cat_acc:", val_cat_acc)
print("val_top_2_acc:", val_top_2_acc)
print("val_top_3_acc:", val_top_3_acc)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  # Required for the heatmap
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import top_k_categorical_accuracy
from sklearn.metrics import classification_report, confusion_matrix # Added confusion_matrix

base_path = "/content/drive/MyDrive/btech_project"
saved_model_path = os.path.join(base_path, "eff_fe_ov1_phase3.keras")

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

print(f"Loading model from: {saved_model_path}...")

model = load_model(
    saved_model_path,
    custom_objects={
        'top_2_accuracy': top_2_accuracy,
        'top_3_accuracy': top_3_accuracy
    }
)
print("Evaluating on training set... (this may take a minute)")

# Evaluate returns [loss, accuracy, top_2, top_3] based on your compile step
results = model.evaluate(train_gen, verbose=1)

print(f"Final Training Loss: {results[0]:.4f}")
print(f"Final Training Accuracy: {results[1]:.4f}")

val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = model.evaluate(val_gen)
print("val_loss:", val_loss)
print("val_cat_acc:", val_cat_acc)
print("val_top_2_acc:", val_top_2_acc)
print("val_top_3_acc:", val_top_3_acc)


test_loss, test_cat_acc, test_top_2_acc, test_top_3_acc = model.evaluate(test_gen)

print("\n📌 TEST METRICS")
print("Test loss:", test_loss)
print("Test categorical accuracy:", test_cat_acc)
print("Test top-2 accuracy:", test_top_2_acc)
print("Test top-3 accuracy:", test_top_3_acc)

# ---------------------------------------------------------
# 4. Predict on the test generator
# ---------------------------------------------------------
print("Generating predictions...")
pred_probs = model.predict(test_gen)
y_pred = np.argmax(pred_probs, axis=1)

# Extract true labels (Keras stores them here)
y_true = test_gen.classes     # integer labels from generator

# ---------------------------------------------------------
# 5. Map indices to class names
# ---------------------------------------------------------
idx_to_class = {v: k for k, v in test_gen.class_indices.items()}
class_names = [idx_to_class[i] for i in range(len(idx_to_class))]

print("\nClass indices mapping:", test_gen.class_indices)
print("Class names in order:", class_names)


print("\n📌 CLASSIFICATION REPORT (HAM10000)\n")
print(classification_report(
    y_true,
    y_pred,
    target_names=class_names,
    digits=2
))


print("\n📌 CONFUSION MATRIX\n")

# Compute the matrix
cm = confusion_matrix(y_true, y_pred)

# Plotting the Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()